In [1]:
from datetime import datetime, timedelta

import pandas as pd

from util import create_spotify_playlist

df = pd.read_parquet("../data/listening_history_with_internet_data.parquet")

In [ ]:
# My top 10 tracks from Frank Ocean by hours played
results = (
    df[df['artist_name'] == "Frank Ocean"]
    .groupby('track_id')
    .agg(
        track_name=('track_name', 'first'),
        album_name=('album_name', 'first'),
        hours_played=('hours_played', lambda x: x.sum().round(2)),
        full_play_count=('full_play', 'sum'),
        skip_count=('full_play', lambda x: (~x).sum()),
    )
    .reset_index()
    .drop(columns=['track_id'])
    .sort_values('hours_played', ascending=False)
    .head(10)
)
results

,track_name,album_name,hours_played,full_play_count,skip_count
10,Nikes,Blonde,8.04,84,33
64,Nights,Blonde,6.52,70,59
17,DHL,DHL,5.64,67,24
20,Ivy,Blonde,4.89,60,49
19,White Ferrari,Blonde,4.55,56,36
52,Chanel,Chanel,3.70,57,31
46,Futura Free,Blonde,2.95,2,41
34,Pink + White,Blonde,2.89,48,47
41,Novacane,Novacane,2.74,28,25
29,Lost,channel ORANGE,2.66,33,35


In [95]:
# My top 10 tracks from female artists by hours played
results = (
    df[df['gender'] == "female"]
    .groupby('track_id')
    .agg({
        'track_name': 'first',
        'artist_name': 'first',
        'album_name': 'first',
        'album_release_year': 'first',
        'citizenship': 'first',
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .drop(columns=['track_id'])
    .sort_values('hours_played', ascending=False)
    .head(10)
)
results

,track_name,artist_name,album_name,album_release_year,citizenship,hours_played,full_play
82,Brooklyn Baby,Lana Del Rey,Ultraviolence,2014,United States,4.52,40
2005,Video Games,Lana Del Rey,Born To Die,2012,United States,4.02,46
890,Florida Kilos,Lana Del Rey,Ultraviolence,2014,United States,3.56,45
2302,meta angel,FKA twigs,CAPRISONGS,2022,United Kingdom,3.21,42
56,This Is What Makes Us Girls,Lana Del Rey,Born To Die,2012,United States,2.77,36
2737,Cruel World,Lana Del Rey,Ultraviolence,2014,United States,2.73,22
1778,Dark Paradise,Lana Del Rey,Born To Die,2012,United States,2.61,36
781,Sad Girl,Lana Del Rey,Ultraviolence,2014,United States,2.60,26
707,Run the World (Girls),Beyoncé,4,2011,United States,2.58,36
2224,Old Money,Lana Del Rey,Ultraviolence,2014,United States,2.55,27


In [4]:
# My top tracks from female artists by hours played (one per artist)
results = (
    df[df['gender'] == "female"]
    .groupby('track_id')
    .agg({
        'track_name': 'first',
        'artist_name': 'first',
        'album_name': 'first',
        'album_release_year': 'first',
        'citizenship': 'first',
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .sort_values('hours_played', ascending=False)
    # Group by artist_name and take the top track for each
    .groupby('artist_name')
    .first()
    .reset_index()
    .reindex(columns=['artist_name', 'track_name', 'album_name', 'album_release_year', 'citizenship', 'hours_played', 'full_play', 'track_id'])
    .sort_values('hours_played', ascending=False)
    .head(10)
)
results

,artist_name,track_name,album_name,album_release_year,citizenship,hours_played,full_play,track_id
262,Lana Del Rey,Brooklyn Baby,Ultraviolence,2014,United States,4.52,40,0CuQKW9YZZA3vcnfpKLCSj
146,FKA twigs,meta angel,CAPRISONGS,2022,United Kingdom,3.21,42,6EeuY84I1Q3UJvpPN6iSzw
55,Beyoncé,Run the World (Girls),4,2011,United States,2.58,36,1uXbwHHfgsXcUKfSZw5ZJ0
229,Kali Uchis,Speed,Por Vida,2015,Colombia,2.43,31,0JmO3EBtl9qhnLkzr4FiP0
290,M.I.A.,Foreign Friend,AIM (Deluxe),2016,Sri Lanka,2.27,29,35yMvnAR2nrhhgWVxkxQON
138,Empress Of,Woman Is a Word,Woman Is a Word,2016,United States,2.06,36,1RhyVwNEpuxHgqpHP686v7
445,Tove Lo,Glad He's Gone,Sunshine Kitty,2019,Sweden,1.93,33,3PUIguYpwKFXmfGD6uU7Jl
208,Jetta,I'd Love To Change The World - Matstubs Remix,I'd Love To Change The World (Matstubs Remix),2015,United Kingdom,1.78,29,41oWiFvBdeS6srKVl20hmt
397,Santigold,Who Be Lovin' Me (feat. ILOVEMAKONNEN),99 Cents,2015,United States,1.65,24,3QJ8oFG4wmUNUnMDoAbvzK
156,Fleur East,Sax - Wideboys Remix,Sax (The Selection),2015,United Kingdom,1.62,21,5u85K4yttujQrvnjvDCLGN


In [14]:
# My top tracks from female artists by hours played (one per artist)
three_years_ago = datetime.now() - timedelta(days=3*365)
results = (
    df[(df['gender'] == "female") & (df['ts'] >= three_years_ago)]
    .groupby('track_id')
    .agg({
        'track_name': 'first',
        'artist_name': 'first',
        'album_name': 'first',
        'album_release_year': 'first',
        'citizenship': 'first',
        'gender': 'first',
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .sort_values('hours_played', ascending=False)
    # Group by artist_name and take the top track for each
    .groupby('artist_name')
    .first()
    .reset_index()
    .reindex(columns=['artist_name', 'track_name', 'album_name', 'album_release_year', 'citizenship', 'gender', 'hours_played', 'full_play', 'track_id'])
    .sort_values('hours_played', ascending=False)
    .head(20)
)
results

,artist_name,track_name,album_name,album_release_year,citizenship,gender,hours_played,full_play,track_id
52,FKA twigs,meta angel,CAPRISONGS,2022,United Kingdom,female,3.07,40,6EeuY84I1Q3UJvpPN6iSzw
55,Fuffifufzich,Zauberstaub,Heartbreakerei,2022,Germany,female,1.14,21,4jktWufGaen0TytRGDjbjK
87,Lana Del Rey,Chemtrails Over The Country Club,Chemtrails Over The Country Club,2021,United States,female,1.12,11,7bPWdJgx8vek7S5i5yAtvG
28,Caroline Polachek,Hit Me Where It Hurts,Pang,2019,United States,female,1.12,19,2wcrQZ7ZJolYEfIaPP9yL4
153,Tove Lo,Glad He's Gone,Sunshine Kitty,2019,Sweden,female,1.09,19,3PUIguYpwKFXmfGD6uU7Jl
17,Baby B3ns,Baby Blizzard,Baby Blizzard,2022,<NA>,female,1.02,28,3y0j1dKTB0XXJQPRNIEXgN
112,Nicki Minaj,Ganja Burn,Queen (Deluxe),2018,Trinidad and Tobago,female,0.95,10,5i8ZRP7tOHPsFBTPU7GXNO
110,Nelly Furtado,All Good Things (Come To An End),Loose (Expanded Edition),2021,Portugal,female,0.82,4,0H9QStpQ5xwHPnVKmqloGk
43,Domiziana,Hello Kitty / Elfbar,Hello Kitty / Elfbar,2023,Germany,female,0.82,15,0J7IXRkMli2i5ptKDGlaJk
26,Cardi B,Get Up 10,Invasion of Privacy,2018,United States,female,0.80,11,7p2ewixAShLpjDZrnzZK7c


In [ ]:
track_ids = results["track_id"].tolist()
if False: # dont run this automatically, so it doesn't create a new playlist every time
    create_spotify_playlist(track_ids, "my top tracks from female artists in the last 3 years")

In [20]:
# Get artists without wikidata info, sorted by hours played
results = (
    df[~df["wikidata_entity_id"].notna()]
    .groupby("artist_id")
    .agg({
        "artist_name": "first",
        "hours_played": "sum",
        "artist_popularity": "first",
        "wikidata_entity_id": "first",
    })
    .reset_index()
    .sort_values("hours_played", ascending=False)
)
results[0:10]

,artist_id,artist_name,hours_played,artist_popularity,wikidata_entity_id
2633,6xI7gsyNgmODoMoWnVp8QG,Oy,16.710186,9,<NA>
1984,5HZXZ01UBUXhMpdei2vULP,Subculture Sage,15.585530,27,<NA>
255,0dnGb3kfZUPdZnwjn7tj98,MoooN,12.740580,3,<NA>
875,2H348ORtPqCyeJllhI8ddB,Al Pride,12.515845,24,<NA>
2187,5pEzNHQvHWJ3f3Z61w1g93,Dan San,11.008588,21,<NA>
707,1n3X60xWCyL1zytSiKeu4D,St. South,10.550475,37,<NA>
5,01TQ6CLvPSOYGUqRQ3nWgZ,Geowulf,9.909039,40,<NA>
1034,2gIGkRqCo4NK2PUCI8u4aQ,Ferdinand fka Left Boy,9.501217,49,<NA>
2678,74yjU6gHx05PeHAokao6DS,NUGAT,9.438550,38,<NA>
387,0xZJXxDyRUTfcsDJpIC5Is,Darjeeling,8.965634,2,<NA>


In [19]:
# Get artists without wikidata info, sorted by popularity
results = (
    df[~df["wikidata_entity_id"].notna()]
    .groupby("artist_id")
    .agg({
        "artist_name": "first",
        "artist_popularity": "first",
        "wikidata_entity_id": "first",
    })
    .reset_index()
    .sort_values("artist_popularity", ascending=False)
)
results[0:10]

,artist_id,artist_name,artist_popularity,wikidata_entity_id
1549,47mIJdHORyRerp4os813jD,League of Legends,81,<NA>
596,1VJ0briNOlXRtJUAzoUJdt,FISHER,76,<NA>
1186,35WVTyRnKAoaGExqgktVyb,Hotel Ugly,73,<NA>
698,1l6d0RIxTL3JytlLGvWzYe,Benjamin Blümchen,72,<NA>
881,2ICR2m4hOBPhaYiZB3rnLW,Sonder,71,<NA>
2094,5b5bt4mZQpJMoCRbiQ7diH,Royel Otis,70,<NA>
1768,4hFBhdNVZZuVk5FYThUwaN,アトラスサウンドチーム,69,<NA>
13,027TpXKGwdXP7iwbjUSpV8,The Walters,69,<NA>
2314,69tiO1fG8VWduDl3ji2qhI,Mt. Joy,69,<NA>
494,1E5hfn5BduN2nnoZCJmUVG,Victony,68,<NA>


In [96]:
# My top 10 tracks from solo artists from 1950-1970
results = (
    df[(df['is_band'] == False) & (df['album_release_year'] >= 1950) & (df['album_release_year'] <= 1970)]
    .groupby('track_id')
    .agg({
        'track_name': 'first',
        'artist_name': 'first',
        'album_name': 'first',
        'album_release_year': 'first',
        'citizenship': 'first',
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .drop(columns=['track_id'])
    .sort_values('hours_played', ascending=False)
    .head(10)
)
results

,track_name,artist_name,album_name,album_release_year,citizenship,hours_played,full_play
95,Peace Piece,Bill Evans,Everybody Digs Bill Evans,1959,United States,0.81,4
69,My Way,Frank Sinatra,My Way (Expanded Edition),1969,Italy,0.50,5
80,Fever,Peggy Lee,All Aglow Again!,1960,United States,0.23,4
127,Un monumento,Ennio Morricone,I Crudeli - The Hellbenders (Original Motion P...,1967,Italy,0.20,4
1,Some Velvet Morning,Nancy Sinatra,Nancy & Lee,1968,United States,0.18,3
38,The Folks,Dave Grusin,The Graduate,1968,United States,0.18,4
134,Space Oddity - 2015 Remaster,David Bowie,David Bowie (aka Space Oddity) [2015 Remaster],1969,United Kingdom,0.18,2
114,Season of the Witch,Donovan,Sunshine Superman,1966,United Kingdom,0.17,2
46,Summertime,Ella Fitzgerald,Porgy And Bess,1958,United States,0.17,2
11,Astral Weeks - 1999 Remaster,Van Morrison,Astral Weeks,1968,United Kingdom,0.16,1


In [97]:
# My top 10 tracks from artists with 'classical' in their genre
results = (
    df[df['artist_genres'].str.contains('classical', case=False, na=False)]
    .groupby('track_id')
    .agg({
        'track_name': 'first',
        'artist_name': 'first',
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .drop(columns=['track_id'])
    .sort_values('full_play', ascending=False)
    .head(10)
)
results

,track_name,artist_name,hours_played,full_play
274,Spirited Away - One Summer's Day,Joe Hisaishi,2.23,21
188,Spirited Away - Reprise,Joe Hisaishi,1.32,15
217,Mystery of Love,Sufjan Stevens,0.85,12
272,Spirited Away - The Sixth Station,Joe Hisaishi,0.79,12
6,Improvisació 1,Bobby McFerrin,2.04,10
25,VIII. Juliet,Matthew Bourne,0.42,7
150,"An Ending, a Beginning",Dustin O'Halloran,0.30,7
43,Ambre,Nils Frahm,0.32,5
374,Spirited Away - The Dragon Boy / The Bottomles...,Joe Hisaishi,0.66,5
91,Premier mouvement - From Nausicaä dans la vall...,London Philharmonic Orchestra,0.63,4


In [100]:
results = (
    df
    .groupby('citizenship')
    .agg({
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .sort_values('full_play', ascending=False)
    .head(10)
)
results

,citizenship,hours_played,full_play
96,United States,1544.07,20447
32,Germany,322.42,4615
94,United Kingdom,208.97,2589
29,France,62.13,927
64,New Zealand,53.97,784
13,Canada,54.71,780
16,Colombia,32.65,516
86,Sweden,34.08,458
85,Sri Lanka,31.66,416
6,Austria,20.11,336


In [101]:
results = (
    df
    .groupby('gender')
    .agg({
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .sort_values('full_play', ascending=False)
    .head(10)
)
results

,gender,hours_played,full_play
3,male,2063.89,27904
0,female,556.05,7636
1,genderfluid,14.13,180
4,non-binary,11.34,150
5,trans man,2.32,37
6,trans woman,0.59,12
7,two-spirit,0.32,6
2,genderqueer,0.09,1
8,undisclosed gender,0.08,0


In [102]:
results = (
    df
    .groupby('is_band')
    .agg({
        'hours_played': lambda x: x.sum().round(2),
        'full_play': 'sum',
    })
    .reset_index()
    .sort_values('full_play', ascending=False)
    .head(10)
)
results

,is_band,hours_played,full_play
0,False,2646.43,35897
1,True,2071.83,27678
